---
###  google colab 버전 
* 이번 12차 수업 자료는 goole colab 버전으로 제공됩니다.
* 구글 드라이브에 올려서 작업하세요
---

### AI HUB 에서 안면 인식 데이터 다운로드
* https://aihub.or.kr/aidata/34143 에서 안면인식 데이터를 다운로드 받습니다
 ![image](https://aihub.or.kr/sites/default/files/2021-06/2020-02-114_%EC%95%88%EB%A9%B4%EC%9D%B8%EC%8B%9D%20%EC%98%81%EC%83%81.png)
* 위 그림과 같은 폴더 구조이고 원천 데이터는 Training 폴더 아래에 17개의 zip 파일과 Validation 폴더 아래에 2개의 zip 파일 형태로 압축되어 있습니다.
* 총 용량이 약 400G가 넘기 때문에 goole drive desktop 설치 등으로 다운받을 디스크 용량을 먼저 확보하세요. 
  * 구글 드라이브 데스크탑은 kookmin.ac.kr 계정이 있으면 https://dl.google.com/drive-file-stream/GoogleDriveSetup.exe 에서 다운로드 받아 설치 가능할 것으로 예상됩니다.
* 디스크 확보가 어려운 경우 AI HUB 의 데이터 다운로드 대화창에서 전체 선택이 아닌 일부 선택을 해서 일부 파일만 다운받는 것도 가능합니다
  ![image](https://user-images.githubusercontent.com/1628848/142525201-86197340-9356-4b7f-84a0-d4bc820306df.png)

---
### 압축 파일 드라이브 특정 위치로 옮기기
* 아래 코드는  한 사람당 100개의 이미지를 랜덤하게 뽑아서 얼굴 인식 학습에 쓸 데이터를 준비하는 과정입니다.
* 아래 코드의 첫 번쩨 셀을 분석한 후 ZIP_DIR에 위에서 다운받은 zip 파일들을 옮겨 넣으세요
* 상당히 오래 걸리니 테스트로 한두개의 zip 으로 정상 동작 여부 먼저 테스트 해 보세요.  한두개의 zip 이어도 3~5시간 가량 소요될 수 있습니다. 

### 아래 코드를 전체 실행 하세요
* 아래 코드를 전체 실행하세요. 수 시간 걸립니다.
* 코드 분석 후 추출된 이미지가 저장되는 폴더에서 이미지가 잘 추출되어 쌓이고 있는지 확인하세요. 


In [ ]:
import shutil
from pathlib import Path
from glob import glob
import random

from tqdm.auto import tqdm

from google.colab import drive
drive.mount('/content/gdrive')

ROOT_DIR = '/content/gdrive/MyDrive/기계학습프로젝트/12차시'
ZIP_DIR = f'{ROOT_DIR}/zip_files'
UNZIP_DIR = f'{ROOT_DIR}/unzip_files'

!ls -l $ROOT_DIR
!ls -l $ZIP_DIR

In [ ]:
# zip 압축 파일 리스팅
zip_files = glob(f'{ZIP_DIR}/*.zip')
zip_files

In [ ]:
# zip 압축 파일 풀기
for zip_file in tqdm(zip_files):
    !unzip $zip_file -d $UNZIP_DIR


In [ ]:
# zip 압축 풀려진 결과물 .tar 파일 리스팅
tars = glob(f'{UNZIP_DIR}/*/*.tar')
len(tars), tars[0]

In [ ]:
# tar 파일에서 이미지를 추출하는  함수 정의
def extract_imgs_from_tar(tar_path, img_num = 100):

    # 기존 폴더를 지우고 새로 폴더를 만드는 유틸 함수
    def mkdir_clean(path):
        shutil.rmtree(path, ignore_errors=True)
        Path(path).mkdir(parents=True)

    # temp 폴더에 압축을 해재한다
    temp_dir = f'{ROOT_DIR}/tar_temp'
    mkdir_clean(temp_dir)
    !tar xf $tar -C $temp_dir

    # jpg 이미지 리스트 읽어오기
    imgs= f'{temp_dir}/*/*/*/real*/color/image/*.jpg'
    imgs = glob(imgs)
    random.shuffle(imgs)

    # 복사해 넣을 이미지 폴더 생성
    person_id = imgs[0].split('/')[-7]
    person_dir = f'{ROOT_DIR}/images/{person_id}'
    mkdir_clean(person_dir)
    print(person_dir)

    # 이미지와 라벨 파일 복사
    for idx, src in enumerate(tqdm(imgs[:img_num])):
        src2 = src.replace('.jpg', '.json')
        dst = f'{person_dir}/{idx:03d}.jpg'
        dst2 = dst.replace('.jpg', '.json')
        shutil.copy(src, dst)
        shutil.copy(src2, dst2)
    shutil.rmtree(temp_dir, ignore_errors=True)

In [ ]:
# 모든 tar 파일에서 img 파일 추출 후 복사
for tar in tqdm(tars):
    extract_imgs_from_tar(tar)

In [ ]:
# 임시 파일을 삭제한다
print(UNZIP_DIR)
shutil.rmtree(UNZIP_DIR, ignore_errors=True)

In [ ]:
drive.flush_and_unmount()